In [2]:
import pandas as pd

In [3]:
df = pd.read_csv(
    "SMSSpamCollection.txt", delimiter="\t", header=None, names=["spam", "text"]
)


In [4]:
sizeData = len(df.index)
trainSize = int(0.5*sizeData)
trainDF = df.sample(trainSize)
testDF = df.drop(trainDF.index)


In [5]:
trainDFSpam = trainDF[trainDF["spam"]=="spam"]
trainDFSpam["text"]=trainDFSpam["text"].str.lower()
dfSplitSpam = trainDFSpam.text.str.split(expand=True).stack().value_counts().to_frame()

trainDFHam = trainDF[trainDF["spam"]=="ham"]
trainDFHam["text"]=trainDFHam["text"].str.lower()
dfSplitHam = trainDFHam.text.str.split(expand=True).stack().value_counts().to_frame()

total = dfSplitSpam.join(dfSplitHam,lsuffix='_spam', rsuffix='_ham')
total.fillna(0.001,inplace=True)
spamTotal=total["count_spam"].sum()
hamTotal = total["count_ham"].sum()
total["vraisemblance"]=(total["count_spam"]/spamTotal)/(total["count_ham"]/hamTotal)

spamProbality = spamTotal/hamTotal

C:\Users\melos\AppData\Local\Temp\ipykernel_16248\3630862678.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  trainDFSpam["text"]=trainDFSpam["text"].str.lower()
C:\Users\melos\AppData\Local\Temp\ipykernel_16248\3630862678.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  trainDFHam["text"]=trainDFHam["text"].str.lower()


In [6]:
total

,count_spam,count_ham,vraisemblance
to,336,752.000,1.016205
a,190,523.000,0.826251
call,161,123.000,2.977014
your,118,186.000,1.442877
you,107,792.000,0.307269
...,...,...,...
hello.,1,3.000,0.758122
"send,",1,1.000,2.274365
chgs,1,0.001,2274.364679
ok!,1,0.001,2274.364679


In [14]:
df = total.sort_values("vraisemblance", ascending=False).head(40)

In [60]:
from matplotlib.axis import YAxis
import plotly.express as px
import numpy as np
violin = total[(total["vraisemblance"] > 0) & (total["vraisemblance"] < 10)]
px.violin(violin, y="vraisemblance")

In [56]:
total["vraisemblance"].min()

np.float64(0.02555465931639965)

In [7]:
def TestPhraseList(phraseList,spamProba=1):
    for word in phraseList:
        vraisemblance = total["vraisemblance"].get(word,"NotFound")
        if vraisemblance!="NotFound":
            spamProba*=vraisemblance
    return(spamProba)
print(TestPhraseList(["hello","call","to","win","your","prize"]))
print(TestPhraseList(["hello","call","me","i","miss","you"]))

def TestPhrase(phrase:str,spamProba=1):
    phraseList=phrase.split()
    return TestPhraseList(phraseList,spamProba=spamProba)


print(TestPhrase("Hi how are you ? I have a wonderful proposition for you ! Gain your wonderful price by clicking on this link"))


806720.1975777625
9.81663907642344e-05
0.0013991362846537806


In [8]:
testDF["text"]=testDF["text"].str.split()

In [9]:
testDF["result"]=testDF["text"].apply(TestPhraseList,spamProba=0.5)
testDF["Finalresult"]=['spam' if x > 1 else 'ham' for x in testDF['result']]
wrong = len(testDF[testDF["Finalresult"]!=testDF["spam"]].index)
FP = len(testDF[(testDF["Finalresult"]=="spam")&(testDF["spam"]=="ham")].index)
FN = len(testDF[(testDF["Finalresult"]=="ham")&(testDF["spam"]=="spam")].index)
VP = len(testDF[(testDF["Finalresult"]=="ham")&(testDF["spam"]=="ham")].index)
VN = len(testDF[(testDF["Finalresult"]=="spam")&(testDF["spam"]=="spam")].index)
wrong/len(testDF.index)
exactitude = (VN+VP)/(VN+FN+FP+VP)
print(f"Exactitude : {exactitude}")
rappel = VP /(VP+FN)
print(f"Rappel : {rappel}")
precision = VP/(VP+FP)
print(f"Precision : {precision}")
F1Score = (2 * precision * rappel) / (precision + rappel)
print(f"F1-Score : {F1Score}")

Exactitude : 0.9468772433596554
Rappel : 0.9808265871325096
Precision : 0.9571725571725572
F1-Score : 0.9688552188552189
